# Deployment Pipeline

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Run the end to end workflow described in the previous exercise for deploying an Amazon SageMaker Endpoint

***

## Part 1/3 - Setup
Here we'll import some libraries and define some variables.

In [2]:
import boto3
import logging
import os
import sys

In [3]:
sys.path.insert(0, os.path.abspath('./../mlpipelines'))

In [4]:
from deployment.pipeline import get_pipeline

In [5]:
s3_client = boto3.client('s3')
sm_client = boto3.client('sagemaker')

In [6]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

***

## Part 2/2 - Run the Pipeline 

### Explore Pipeline definition

In [7]:
!pygmentize ./../mlpipelines/deployment/pipeline.py

"""
This sample is non-production-ready template
© 2021 Amazon Web Services, Inc. or its affiliates. All Rights Reserved.
This AWS Content is provided subject to the terms of the AWS Customer Agreement available at
http://aws.amazon.com/agreement or other written agreement between Customer and either
Amazon Web Services, Inc. or Amazon Web Services EMEA SARL or both.
"""

import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import logging
from sagemaker import get_execution_role
from sagemaker.model_monitor import DataCaptureConfig
import sagemaker.session
from sagemaker.tensorflow import TensorFlowModel
import traceback

logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

sagemaker_client = boto3.client("sagemaker")

def get_kms_key(
        region,
        account_id,
        kms_alias):
    try:
        kms_key = "arn:aws:kms:{}:{}:alias/{}".format(region, account_id, kms_alias)

        return kms_key
    except Exception 

### Pipeline Parameters

In [8]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

env = "dev"

kms_account_id = boto3.client('sts').get_caller_identity().get('Account')

kms_alias = "ml-kms"

bucket_artifacts = ""
bucket_inference = ""

inference_artifact_path = "artifact/inference"
inference_artifact_name = "sourcedir.tar.gz"
inference_instance_count = 1
inference_instance_type = "ml.m5.xlarge"

model_package_group = "ml-end-to-end-group"

monitoring_output_path = "data/monitoring/captured"

training_framework_version = 2.4

### Compress source code for installing additional python modules

In [11]:
! ./../algorithms/buildspec.sh inference $bucket_artifacts

inference.py
requirements.txt
services/
services/.ipynb_checkpoints/
services/.ipynb_checkpoints/TranslateService-checkpoint.py
services/TranslateService.py
services/__init__.py
utils/
utils/utils.py
utils/constants.py
utils/__init__.py
mkdir: cannot create directory ‘../../dist’: File exists
Uploading s3://isengard-bpistone-ml-end-to-end-dev/artifact/inference/sourcedir.tar.gz
upload: ../../dist/inference/sourcedir.tar.gz to s3://isengard-bpistone-ml-end-to-end-dev/artifact/inference/sourcedir.tar.gz


***

### Deployment pipeline

#### Get pipeline definition

In [ ]:
pipeline = get_pipeline(
    region,
    env,
    kms_account_id,
    kms_alias,
    bucket_artifacts,
    bucket_inference,
    inference_artifact_path,
    inference_artifact_name,
    inference_instance_count,
    inference_instance_type,
    model_package_group,
    monitoring_output_path,
    training_framework_version,
    role
)

INFO:deployment.pipeline:Identified the latest approved model package: {'ModelPackageGroupName': 'ml-end-to-end-group', 'ModelPackageVersion': 12, 'ModelPackageArn': 'arn:aws:sagemaker:eu-west-1:691148928602:model-package/ml-end-to-end-group/12', 'CreationTime': datetime.datetime(2022, 6, 9, 21, 23, 32, 121000, tzinfo=tzlocal()), 'ModelPackageStatus': 'Completed', 'ModelApprovalStatus': 'Approved'}
INFO:deployment.pipeline:{'ModelPackageGroupName': 'ml-end-to-end-group', 'ModelPackageVersion': 12, 'ModelPackageArn': 'arn:aws:sagemaker:eu-west-1:691148928602:model-package/ml-end-to-end-group/12', 'CreationTime': datetime.datetime(2022, 6, 9, 21, 23, 32, 121000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.eu-west-1.amazonaws.com/tensorflow-inference:2.4-cpu', 'ImageDigest': 'sha256:9655faca3ac7c26a9583ad05a324ca7a58da4af1c14f3d2daae0d217688356a9', 'ModelDataUrl': 's3://isengard-bpistone-ml-end-to-end-dev/models/tensorflow-training-2022-06-